In [27]:
import polars as pl
import sys, os
from sippy import *
import numpy as np
from sippy import functionset as fset
from sippy import functionsetSIM as fsetSIM
import matplotlib.pyplot as plt
import pandas as pd
import control as cnt
import json
from datetime import datetime, timezone
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler

In [28]:
data_path = '/home/alqua/data/data_vdfs'

In [29]:
start_filter_date = datetime(2024, 1, 9, 00, 00, 0, tzinfo=timezone.utc)
end_filter_date = datetime(2024, 2, 20, 00, 00, 0, tzinfo=timezone.utc)

In [30]:
p1_energy = pl.read_parquet(data_path + '/' +'pump1_power_siso.par')
p3_energy = pl.read_parquet(data_path + '/' +'pump3_power_siso.par')
p4_energy = pl.read_parquet(data_path + '/' +'pump4_power_siso.par')
outflow_df = pl.read_parquet(data_path + '/' +'outflow_miso.par')


In [31]:
sampling_time = '60m'

p1_energy = p1_energy.filter(
                                 (pl.col('time') >= start_filter_date) &
                                   (pl.col('time')<= end_filter_date)
                                 )

p3_energy = p3_energy.filter(
                                 (pl.col('time') >= start_filter_date) &
                                   (pl.col('time')<= end_filter_date)
                                 )

p4_energy = p4_energy.filter(
                                 (pl.col('time') >= start_filter_date) &
                                   (pl.col('time')<= end_filter_date)
                                 )


In [32]:
sysid_df = outflow_df.join(p1_energy, 
                left_on='time', 
                right_on='time').join(p3_energy, 
                left_on='time', 
                right_on='time').join(
                p4_energy, 
                left_on='time', 
                right_on='time').group_by_dynamic(
                        'time', 
                        every=sampling_time, 
                        ).agg(pl.all().mean())

In [33]:
features_cols = {
    'time': 'time_utc',  
    'outflow': 'qout',
    'pump1_power': 'p1_energy',
    'pump3_power': 'p3_energy',
    'pump4_power': 'p4_energy',
}

sysid_df = (
    sysid_df
    .select(features_cols.keys())
    .rename(features_cols)
    )


In [34]:
from sippy import functionset as fset

def identify_system(df, u_col, y_col, test_size=0.1, na=1, nb=2, theta=0, dt=None, predict_test =False, nsteps_ahead=1, plot_results=False, save_tf=False):

    selected_data = df.select([u_col, y_col])
    split_point = int(len(selected_data) * test_size)
    train_df = selected_data.head(split_point)
    test_df = selected_data.tail(len(selected_data) - split_point) 
    


    u_train, u_test = train_df[u_col].to_numpy(), test_df[u_col].to_numpy()
    y_train, y_test = train_df[y_col].to_numpy(), test_df[y_col].to_numpy()
    
    na_ords = [na]         
    nb_ords = [[nb]]       
    theta = [[theta]] 

    id_ARX = system_identification(y_train, u_train, 'ARX', 
                                   stab_cons=True, SS_A_stability=True,
                                   #centering='MeanVal',
                                   ARX_orders=[na_ords, nb_ords, theta],
                                    tsample=dt) 
    
    G = id_ARX.G  
    print(f"\nTransfer function from {u_col} to {y_col}:")
    print("==================")
    print(id_ARX.G)
    if save_tf: 
        tf_data = {
                    "u": u_col,
                    "y": y_col,
                    "na": na, 
                    "nb":nb,
                    "num": [round(x, 5) for x in id_ARX.NUMERATOR[0][0]],
                    "den": [round(x, 5) for x in id_ARX.DENOMINATOR[0][0][1:]],
                    "dt": dt
                }
        
        filename = f"tf_{u_col}_to_{y_col}.json"
        result_path = "results/"
        with open(result_path + filename, 'w') as f:
            json.dump(tf_data, f, indent=4)

    if predict_test: 
        t_test = np.arange(0, len(y_test)) * dt
        Yval = fset.validation(id_ARX, u_test, y_test, t_test, k=nsteps_ahead)
        if plot_results: 
            fig = FigureWidgetResampler(go.Figure())
            fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))
            fig.add_trace(
                        go.Scattergl(
                            x=t_test,
                            y=y_test,
                            name=f'{y_col} (Predicted from {u_col})',  # Shows column names
                            showlegend=True,
                            mode='lines'
                        )
                    )
            fig.add_trace(
                        go.Scattergl(
                            x=t_test,
                            y=Yval.flatten(),
                            name=f'{y_col} (Predicted from {u_col})',  # Shows column names
                            showlegend=True,
                            mode='lines'
                        )
                    )
            fig.update_layout(height=200, template="plotly_dark")
            display(fig)
            
        
        return id_ARX, G, t_test, Yval

    
    else: 
        return id_ARX, G


In [35]:
sysid_df = sysid_df.with_columns((pl.col("p1_energy") + pl.col("p3_energy") + pl.col("p4_energy")).alias("cum_energy"))

In [36]:
sysid_df = sysid_df.with_columns(pl.col("cum_energy").shift(-3)).drop_nulls()

In [37]:
columns = [
    ("cum_energy", "qout")
]


for u_col, y_col in columns:
    print(f"Identifying model {u_col}, {y_col}")
    id_ARX, G, t_test, Yval = identify_system(
        df=sysid_df, 
        u_col=u_col,
        y_col=y_col,
        test_size=0.5, 
        na=1,
        nb=1,
        theta=1,
        #dt=int(sampling_time[:2]),
        dt=1,
        predict_test=True, 
        nsteps_ahead=1, 
        plot_results=True, 
        save_tf=True)


Identifying model cum_energy, qout

Transfer function from cum_energy to qout:

    4.647
--------------
z^2 - 0.5336 z

dt = 1



FigureWidgetResampler({
    'data': [{'mode': 'lines',
              'name': 'qout (Predicted from cum_energy)',
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'cebf387f-1229-48fc-9406-960ea1229d11',
              'x': array([  0,   1,   2, ..., 452, 453, 454]),
              'y': array([584.05027778, 576.53027778, 582.23111111, ..., 712.25027778,
                          717.42333333, 736.98314607])},
             {'mode': 'lines',
              'name': 'qout (Predicted from cum_energy)',
              'showlegend': True,
              'type': 'scattergl',
              'uid': '6676f71f-da9f-4a9a-b592-9498bc77db79',
              'x': array([  0,   1,   2, ..., 452, 453, 454]),
              'y': array([  0.        , 311.67253683, 542.23702007, ..., 747.95055341,
                          775.28666714, 779.29775018])}],
    'layout': {'height': 200, 'margin': {'b': 10, 'l': 10, 'r': 10, 't': 10}, 'template': '...'}
})